# Pictures

In [1]:
import cv2            
import pandas as pd  
import numpy as np   
from ultralytics import YOLO  
from tracker import *         

img = cv2.imread('pictures/soccer_practice.jpg')

# Load YOLO object detection model
model = YOLO('yolov8n.pt')

# Load the class names from a file (assuming coco.txt contains class names)
with open("coco.txt", "r") as my_file:
    class_list = my_file.read().split("\n")

# Create a dictionary to store information about detected people
people = {}

tracker = Tracker()

# Use the YOLO model to predict objects in the image
results = model.predict(img)

# Extract bounding box information from the prediction results
a = results[0].boxes.data
px = pd.DataFrame(a).astype("float")

# List to store coordinates of detected people's bounding boxes
detected_people = []

# Loop through each bounding box and its associated data
for index, row in px.iterrows():
    x1 = int(row[0])
    y1 = int(row[1])
    x2 = int(row[2])
    y2 = int(row[3])
    d = int(row[5])
    c = class_list[d]
    
    # Check if the detected object is a person
    if 'person' in c:
        detected_people.append([x1, y1, x2, y2])

num_people = len(detected_people)

# Update the custom object tracker with the detected people's bounding boxes
bbox_idx = tracker.update(detected_people)

# Loop through each tracked bounding box and its associated ID
for bbox in bbox_idx:
    x4, y4, x5, y5, id = bbox
    
    # Draw a rectangle around the tracked person's bounding box
    cv2.rectangle(img, (x4, y4), (x5, y5), (255, 200, 0), 2)
    
    # Display the person's ID near the bounding box
    cv2.putText(img, str(id), (x4, y4 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 200, 0), 2)

# Display the total count of detected people on the image
cv2.putText(img, f'People Count: {num_people}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (250, 255, 0), 2)

cv2.imshow('img', img)

cv2.waitKey(0)


Ultralytics YOLOv8.0.33  Python-3.8.10 torch-2.0.1+cpu CPU
YOLOv8n summary (fused): 168 layers, 3151904 parameters, 0 gradients, 8.7 GFLOPs

0: 608x640 5 persons, 1 sports ball, 530.5ms
Speed: 8.0ms pre-process, 530.5ms inference, 8.0ms postprocess per image at shape (1, 3, 640, 640)


-1

# Videos

In [3]:
import cv2  
import pandas as pd  
import numpy as np  
from ultralytics import YOLO  
from tracker import *  

cap = cv2.VideoCapture('pictures/vid2.mp4')

# Load the YOLO object detection model
model = YOLO('yolov8n.pt')

# Read the COCO class labels from a file
with open("coco.txt", "r") as my_file:
    class_list = my_file.read().split("\n")

# Initialize dictionaries and variables for tracking
people = {}  
tracker = Tracker()  
count = 0  

# Main loop for video processing
while True:

    ret, frame = cap.read()

    count += 1

    # Skip every other frame (frame downsampling)
    if count % 2 != 0:
        continue

    frame = cv2.resize(frame, (1020, 500))

    # Use YOLO model to predict objects in the frame
    results = model.predict(frame)
    
    # Extract bounding box information from the YOLO results
    a = results[0].boxes.data
    px = pd.DataFrame(a).astype("float")

    # Identify detected people and store their bounding box coordinates
    detected_people = []
    for index, row in px.iterrows():
        x1 = int(row[0])
        y1 = int(row[1])
        x2 = int(row[2])
        y2 = int(row[3])
        d = int(row[5])
        c = class_list[d]
        if 'person' in c:
            detected_people.append([x1, y1, x2, y2])
            
    # Count the number of detected people
    num_people = len(detected_people)  

    # Update the object tracker with the current set of detected people
    bbox_idx = tracker.update(detected_people)  
    
    # Draw bounding boxes for tracked people on the frame
    for bbox in bbox_idx:
        x4, y4, x5, y5, id = bbox
        cv2.rectangle(frame, (x4, y4), (x5, y5), (255, 200, 0), 2)
        
    # Display the count of detected people on the frame
    cv2.putText(frame, f'People Count: {num_people}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (250, 255, 0), 2)

    cv2.imshow("RGB", frame)
    
    # Break the loop if the 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Ultralytics YOLOv8.0.33  Python-3.8.10 torch-2.0.1+cpu CPU
YOLOv8n summary (fused): 168 layers, 3151904 parameters, 0 gradients, 8.7 GFLOPs

0: 320x640 32 persons, 2 birds, 174.7ms
Speed: 0.0ms pre-process, 174.7ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)

0: 320x640 31 persons, 2 birds, 180.1ms
Speed: 0.0ms pre-process, 180.1ms inference, 5.0ms postprocess per image at shape (1, 3, 640, 640)

0: 320x640 31 persons, 3 birds, 180.0ms
Speed: 0.0ms pre-process, 180.0ms inference, 3.5ms postprocess per image at shape (1, 3, 640, 640)

0: 320x640 34 persons, 2 birds, 179.2ms
Speed: 0.0ms pre-process, 179.2ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 320x640 33 persons, 2 birds, 169.8ms
Speed: 0.0ms pre-process, 169.8ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 320x640 28 persons, 3 birds, 172.2ms
Speed: 0.0ms pre-process, 172.2ms inference, 5.0ms postprocess per image at shape (1, 3, 640, 640)

0: 320x640 29 perso